In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [0]:
import os
import json

from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from pyspark.sql import SQLContext

from edm.batch_ingestion.param_config_parser import ParamConfigParser
from edm.batch_ingestion.landing_to_raw.landing_to_raw_processor import LandingToRawProcessor
from edm.utils.keyvault_utility import KeyvaultSecretsUtility

In [0]:
spark = SparkSession.builder.appName('').getOrCreate()
dbutils = DBUtils(spark)
sqlContext = SQLContext(spark)

In [0]:
dbutils.widgets.text('source_file_metadata', '')
dbutils.widgets.text('spn_id', '')
dbutils.widgets.text('spn_password', '')
dbutils.widgets.text('tenant_id', '')
dbutils.widgets.text('kv_name', '')
dbutils.widgets.text('adls_account_name', '')
dbutils.widgets.text('config_path', '')

In [0]:
source_file_metadata = json.loads(dbutils.widgets.get('source_file_metadata').replace('\\', ""))
spn_id = dbutils.widgets.get('spn_id')
spn_password = dbutils.widgets.get('spn_password')
tenant_id = dbutils.widgets.get('tenant_id')
kv_name = dbutils.widgets.get('kv_name')
adls_account_name = dbutils.widgets.get('adls_account_name')
config_path = dbutils.widgets.get('config_path')

In [0]:
# spn_password = os.getenv('SPN_PASSWORD', None)

In [0]:
# spn_password = '_o_kU5-MEMVC~Q~AO9dH4Wr2E5-ujcjC6d'
# print(spn_password)

_o_kU5-MEMVC~Q~AO9dH4Wr2E5-ujcjC6d

In [0]:
spn_credentials = dict()
spn_credentials['spn_id'] = spn_id
spn_credentials['spn_password'] = spn_password
spn_credentials['tenant_id'] = tenant_id

In [0]:
for item in source_file_metadata:
  # Param Config Parser
  print(item)
  parser_obj = ParamConfigParser(
      item['Source'], item['Country'], item['FileName'],
      spark, dbutils, spn_credentials=spn_credentials,
      kv_name=kv_name, adls_account_name=adls_account_name,
      config_path=config_path
  )
  parser_obj.run()
  # Landing to Raw
  landing_obj = LandingToRawProcessor(
          item['Source'], item['Country'], item['FileName'], item['PipelineLogId'],
          spark, sqlContext, dbutils, spn_credentials=spn_credentials,
          kv_name=kv_name, adls_account_name=adls_account_name,
          config_path=config_path
      )
  landing_obj.run()

{'Source': 'consumption', 'Country': 'all', 'FilePath': 'incoming/data/consumption/all', 'FileName': 'consumption_all_Sheet1$_20210512.csv'}
15077959 | 2021-07-12 09:34:14,231 | INFO | MainThread | edm.batch_ingestion.param_config_parser: 481 run(): Starting config Param Parser
15077959 | 2021-07-12 09:34:14,231 | INFO | MainThread | edm.batch_ingestion.param_config_parser: 482 run(): Started Initialisation
15079937 | 2021-07-12 09:34:16,209 | INFO | MainThread | edm.batch_ingestion.param_config_parser: 487 run(): Ended Initialisation
15079937 | 2021-07-12 09:34:16,209 | INFO | MainThread | edm.batch_ingestion.param_config_parser: 488 run(): Starting Parsing XML Files
15080029 | 2021-07-12 09:34:16,301 | INFO | MainThread | edm.batch_ingestion.param_config_parser: 449 _get_file_row_list(): No Config Files Found in Config Folder
15080030 | 2021-07-12 09:34:16,302 | INFO | MainThread | edm.batch_ingestion.param_config_parser: 257 parse_param_config(): No Param and Config Files in Blob Account.
15080030 | 2021-07-12 09:34:16,302 | INFO | MainThread | edm.batch_ingestion.landing_to_raw.landing_to_raw_processor: 529 run(): Starting Landing to Raw pipeline
15080030 | 2021-07-12 09:34:16,302 | INFO | MainThread | edm.batch_ingestion.landing_to_raw.landing_to_raw_processor: 530 run(): Started Initialisation
15081285 | 2021-07-12 09:34:17,557 | INFO | MainThread | edm.batch_ingestion.landing_to_raw.landing_to_raw_processor: 535 run(): Ended Initialisation
15081286 | 2021-07-12 09:34:17,558 | INFO | MainThread | edm.batch_ingestion.landing_to_raw.landing_to_raw_processor: 536 run(): Starting Processing Data Files
15083237 | 2021-07-12 09:34:19,509 | INFO | MainThread | edm.batch_ingestion.landing_to_raw.landing_to_raw_processor: 276 landing_to_raw_processor(): Logging to Source File table

--------------------------------------------------------------------------- 
 IndexError Traceback (most recent call last)
 <command-2637393285085592> in <module> 
 16 config_path = config_path
 17 )
 ---> 18 landing_obj . run ( ) 

 /databricks/python/lib/python3.7/site-packages/edm/batch_ingestion/landing_to_raw/landing_to_raw_processor.py in run (self) 
 535 LOGGER . info ( "Ended Initialisation" ) 
 536 LOGGER . info ( "Starting Processing Data Files" ) 
 --> 537 self . landing_to_raw_processor ( ) 

 /databricks/python/lib/python3.7/site-packages/edm/batch_ingestion/landing_to_raw/landing_to_raw_processor.py in landing_to_raw_processor (self) 
 280 '{self.file_name}'"""
 281 )
 --> 282 source_file_details = self . db_obj . run_sql_query ( Query ) [ 0 ] [ 0 ] 
 283 sourcefileprocesslog_id = source_file_details [ 0 ] 
 284 param = {

 IndexError : list index out of range